Let's set up a study workflow using Jupyter Notebooks, LLMs, and langchain.

In [ ]:
# !pip install langchain
# !pip install openai
# !pip install pypdf
# !pip install chromadb
# !pip install langchainhub

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ChatVectorDBChain
from langchain.chat_models import ChatOpenAI
import os
import openai
from dotenv import load_dotenv, find_dotenv
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
persist_directory = "./"
pdf = "./assets-resources/Augmenting-Human-Intellect_ A-Conceptual-Framework-1962-Doug Engelbart.pdf"
loader = PyPDFLoader(pdf)
pdf_doc = loader.load_and_split()
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(pdf_doc, embedding=embeddings,\
    persist_directory=persist_directory)
vectordb.persist()

In [8]:
pdf_qa = ChatVectorDBChain.from_llm(ChatOpenAI(temperature=0,\
    model_name="gpt-4"), vectordb, return_source_documents=True)
query = "What is augmentation according to Douglas Engelbart?"
result = pdf_qa({"question": query, "chat_history": ""})
print("ANSWER")
print(result["answer"])

/Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.11/site-packages/langchain/chains/conversational_retrieval/base.py:405: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warnings.warn(


ANSWER
According to Douglas Engelbart, augmentation refers to the use of computer-based systems to enhance or augment human intellect. This involves the development of new languages, methodologies, and symbol-manipulating capabilities that capitalize on the capabilities of computer-based equipment. The goal is to improve the intellectual capabilities of humans, particularly in problem-solving tasks. This augmentation can involve the use of display screens, keysets, and light pens, among other tools, to interact with the computer system and manipulate symbols in a dynamic and flexible manner.


In [9]:
def ask_pdf(pdf_qa,query):
    print("QUERY: ",query)
    result = pdf_qa({"question": query, "chat_history": ""})
    answer = result["answer"]
    print("ANSWER", answer)
    return answer


ask_pdf(pdf_qa,"What are the four ways through which human capabilities are augmented according to Douglas Engelbart?")

QUERY:  What are the four ways through which human capabilities are augmented according to Douglas Engelbart?
ANSWER I'm sorry, but the provided text does not specify the four ways through which human capabilities are augmented according to Douglas Engelbart.


"I'm sorry, but the provided text does not specify the four ways through which human capabilities are augmented according to Douglas Engelbart."

In [10]:
quiz = ask_pdf(pdf_qa, "Quiz me on theessay with 5 fundamental questions.")

quiz

QUERY:  Quiz me on theessay with 5 fundamental questions.
ANSWER 1. What is the main purpose of the techniques and capabilities demonstrated by Joe in the essay?
2. How does the computer system described in the essay assist in the manipulation and organization of symbol structures?
3. What is the concept of an 'argument' as described in the essay?
4. How does the system help in structuring an argument and enhancing comprehension of complex concepts?
5. What are some of the specific capabilities of the system that Joe demonstrates, such as in relation to text manipulation and linking of substructures?


"1. What is the main purpose of the techniques and capabilities demonstrated by Joe in the essay?\n2. How does the computer system described in the essay assist in the manipulation and organization of symbol structures?\n3. What is the concept of an 'argument' as described in the essay?\n4. How does the system help in structuring an argument and enhancing comprehension of complex concepts?\n5. What are some of the specific capabilities of the system that Joe demonstrates, such as in relation to text manipulation and linking of substructures?"

In [11]:
from openai import OpenAI
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

In [12]:
prompt = f"Return ONLY a PYTHON list containing the questions in this text: {quiz}"

import ast

questions = ast.literal_eval(get_response(prompt))
questions

['What is the main purpose of the techniques and capabilities demonstrated by Joe in the essay?',
 'How does the computer system described in the essay assist in the manipulation and organization of symbol structures?',
 "What is the concept of an 'argument' as described in the essay?",
 'How does the system help in structuring an argument and enhancing comprehension of complex concepts?',
 'What are some of the specific capabilities of the system that Joe demonstrates, such as in relation to text manipulation and linking of substructures?']

In [13]:
answers = []
for q in questions:
    answers.append(ask_pdf(pdf_qa,q))

QUERY:  What is the main purpose of the techniques and capabilities demonstrated by Joe in the essay?
ANSWER The main purpose of the techniques and capabilities demonstrated by Joe in the essay is to show how a computerized system can augment human intellect by making the process of manipulating and organizing information more efficient and flexible. This includes tasks such as composing text, defining abbreviations, and rearranging and editing work. The system allows for rapid and dynamic changes, making it easier to develop and refine ideas. The essay emphasizes that these capabilities, while individually simple, can collectively lead to significant improvements in problem-solving and conceptual thinking.
QUERY:  How does the computer system described in the essay assist in the manipulation and organization of symbol structures?
ANSWER The computer system described in the essay assists in the manipulation and organization of symbol structures in several ways. Firstly, it allows for t

In [14]:
evaluations = []

for q,a in zip(questions, answers):
    # Check for results
    evaluations.append(ask_pdf(pdf_qa,f"Is this: {a} the correct answer to this question: {q} according to the paper? Return only a YES or NO. Output:"))

evaluations

QUERY:  Is this: The main purpose of the techniques and capabilities demonstrated by Joe in the essay is to show how a computerized system can augment human intellect by making the process of manipulating and organizing information more efficient and flexible. This includes tasks such as composing text, defining abbreviations, and rearranging and editing work. The system allows for rapid and dynamic changes, making it easier to develop and refine ideas. The essay emphasizes that these capabilities, while individually simple, can collectively lead to significant improvements in problem-solving and conceptual thinking. the correct answer to this question: What is the main purpose of the techniques and capabilities demonstrated by Joe in the essay? according to the paper? Return only a YES or NO. Output:
ANSWER YES
QUERY:  Is this: The computer system described in the essay assists in the manipulation and organization of symbol structures in several ways. Firstly, it allows for the repres

['YES', 'YES', 'YES', 'YES', 'YES']

In [15]:
scores = []

yes_count = evaluations.count('YES')
scores.append(yes_count)
print(scores)

[5]
